# Hipages Take‑Home Test — Part 1 (Kevin Abraham)

**What this notebook covers**

- **Task 1 — SQL Programming:**  
  1) Names + number of messages sent by each user  
  2) Total messages sent by weekday  
  3) Most recent unanswered message per thread  
  4) Conversation with the most messages (chronological order)
**Brief:** See `Take Home Task - Data Analyst.docx.pdf` in repo.

---

## Approach
- **Data:** Generated fake data using provided ERD
- **SQL:** SQL lite style as running on colab.
- **Python:** Queries are writen in SQL but run using python in pandas dataframes using SQLDF which runs a SQL-lite instance in the background

---

## How to run
- **Data:** Upload into colab files the 3 csv in /datasets. Messages,Threads,Users.csv
- **Run:** Click `Runtime` -> `Run all`

In [27]:
# install packages not already in colab
!pip install pandasql

In [28]:
import pandas as pd # to read the files as a dataset
import pandasql as pdsql # to query the files as sql. Makes it easy to quickly analyse data
import numpy as np # easy calculation library

In [29]:
# read csv files into pandas dataframes
messages = pd.read_csv('Messages.csv', header=0)
threads = pd.read_csv('Threads.csv', header=0)
users = pd.read_csv('User.csv', header=0)

In [30]:
# Q1:The names and the number of messages sent by each user
# Approach: This query shows every user, counts how many messages they’ve sent, even if it’s zero, and then lists them from the most active to the least.
# Code:
q1_1 = """
SELECT users.Name as name,
COUNT(DISTINCT messages.MessageID) AS messages_sent
FROM users
LEFT JOIN messages ON messages.UserIDSender = users.UserID -- includes users who sent 0 messages
GROUP BY users.Name
ORDER BY messages_sent DESC
"""
print("The answer is ↓↓↓")
print(pdsql.sqldf(q1_1))

The answer is ↓↓↓
                name  messages_sent
0        Alex Nguyen             30
1          Sam Smith             27
2        Brook Perez             27
3       Arden Turner             27
4      Blair Roberts             25
5      Remy Campbell             24
6       Hayden Young             24
7    Finley Mitchell             22
8       Jamie Martin             21
9       Sawyer Baker             20
10   Dakota Phillips             20
11      Quinn Walker             19
12       Logan Adams             19
13    Riley Anderson             17
14     Kendall Green             16
15       Drew Wright             16
16      Casey Wilson             16
17       Sage Carter             14
18      Emerson Hall             14
19     Parker Harris             13
20     Harper Nelson             13
21       Reese Allen             12
22     Morgan Taylor             12
23       Rowan Lewis             11
24        Taylor Lee             10
25       Avery White             10
26       E

In [31]:
# Q2 The total number of messages sent stratified by weekday
# Approach: This query counts how many messages were sent on each day of the week and labels them with the weekday name.
# Code:
q1_2 = """
SELECT strftime('%w', DateSent) AS day_of_week,
CASE strftime('%w', DateSent)
  WHEN '0' THEN 'Sun'
  WHEN '1' THEN 'Mon'
  WHEN '2' THEN 'Tue'
  WHEN '3' THEN 'Wed'
  WHEN '4' THEN 'Thu'
  WHEN '5' THEN 'Fri'
  WHEN '6' THEN 'Sat'
END AS weekday_name, -- have to do it this way as SQLlite has no weekday function
COUNT(DISTINCT MessageID) AS messages_sent
FROM messages
GROUP BY day_of_week
ORDER BY CAST(day_of_week AS INTEGER) ASC
"""
print("The answer is ↓↓↓")
print(pdsql.sqldf(q1_2))

The answer is ↓↓↓
  day_of_week weekday_name  messages_sent
0           0          Sun             83
1           1          Mon             85
2           2          Tue            113
3           3          Wed             78
4           4          Thu             32
5           5          Fri             61
6           6          Sat             49


In [32]:
# Q3: The most recent message from each thread that has no response yet
# Approach:
## Take every message in the system.
## Try to find a later message in the same conversation (same thread) that was sent after it.
## If a later message exists, keep track of it.
## If no later message exists, that means the original message has no reply, so count it.
## Sort the results by ThreadID.
# Code
q3 = """
SELECT
  messages.ThreadID,
  messages.MessageID,
  messages.DateSent,
  messages.UserIDSender,
  messages.UserIDRecipient,
  messages.MessageContent,
  later.MessageID AS later_message_id,
  later.MessageContent AS later_MessageContent
FROM messages
LEFT JOIN messages AS later -- left join to get later messages based on response
  ON later.ThreadID = messages.ThreadID
 AND later.DateSent > messages.DateSent -- message must be later
WHERE later.MessageID IS NULL
--AND messages.ThreadID = 1000 -- for QA
ORDER BY messages.ThreadID
"""
print(pdsql.sqldf(q3))

    ThreadID  MessageID             DateSent  UserIDSender  UserIDRecipient  \
0       1000      50005  2025-08-10 18:36:00            18               22   
1       1001      50012  2025-07-29 05:38:00            11                6   
2       1002      50020  2025-07-24 17:02:00             3               19   
3       1003      50026  2025-08-19 09:36:00            25               21   
4       1004      50033  2025-07-31 03:12:00             2                1   
5       1005      50037  2025-08-11 13:21:00             4                1   
6       1006      50046  2025-07-26 07:49:00            26               18   
7       1007      50056  2025-08-07 22:11:00            12                1   
8       1008      50065  2025-08-16 14:59:00            27               25   
9       1009      50074  2025-07-23 06:10:00             2               24   
10      1010      50078  2025-08-13 10:57:00            28               11   
11      1011      50090  2025-08-12 22:18:00        

In [33]:
# Q4: For the conversation with the most messages: all user data and message contents ordered chronologically so one can follow the whole conversation
# Approach:
## Make a clean view of every message with who sent it and who got it (by joining to user names).
## Find which thread has the most messages.
## Number each message in a conversation from first to last using row number (oldest to newest).
## Show only the most_messages conversation and list its messages in order.
# Code
q4 = """
WITH conversations AS (
	SELECT
	  messages.ThreadID,
	  messages.MessageID,
	  messages.DateSent,
	  sender.UserID AS sender_id,
	  sender.Name AS sender_name,
	  receiver.UserID AS receiver_id,
	  receiver.Name AS receiver_name,
	  messages.MessageContent
	FROM messages
	LEFT JOIN users sender ON sender.UserID = messages.UserIDSender
	LEFT JOIN users receiver ON receiver.UserID = messages.UserIDRecipient
),
most_messages AS (
  SELECT ThreadID, COUNT(DISTINCT MessageID) AS cnt
  FROM messages
  GROUP BY 1
  ORDER BY 2 DESC
  LIMIT 1
)
SELECT *
FROM (
  SELECT *,
  ROW_NUMBER() OVER (
    PARTITION BY ThreadID ORDER BY DateSent ASC, MessageID ASC
  ) AS rn
  FROM conversations
)
WHERE ThreadID = (SELECT ThreadID FROM most_messages)
ORDER BY rn ASC


"""
print(pdsql.sqldf(q4))

    ThreadID  MessageID             DateSent  sender_id      sender_name  \
0       1035      50300  2025-07-29 09:26:00         21     Sawyer Baker   
1       1035      50301  2025-07-29 10:08:00         29  Dakota Phillips   
2       1035      50302  2025-07-29 11:17:00         21     Sawyer Baker   
3       1035      50303  2025-07-29 12:12:00         29  Dakota Phillips   
4       1035      50304  2025-07-29 13:13:00         21     Sawyer Baker   
5       1035      50305  2025-07-29 14:31:00         29  Dakota Phillips   
6       1035      50306  2025-07-29 15:27:00         21     Sawyer Baker   
7       1035      50307  2025-07-29 15:52:00         29  Dakota Phillips   
8       1035      50308  2025-07-29 17:19:00         21     Sawyer Baker   
9       1035      50309  2025-07-29 18:00:00         29  Dakota Phillips   
10      1035      50310  2025-07-29 19:23:00         21     Sawyer Baker   
11      1035      50311  2025-07-29 20:28:00         29  Dakota Phillips   
12      1035